[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 130

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [3]:
# Load JSON files and store the raw data
with open("vaccinatedpeople.json", "rt") as INFILE:
    data = json.load(INFILE)

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [4]:
def wrangle_data(data):
    #Get the data from JSON file or API call and sort the date
    datalist=data['body']
    dates=[i['date'] for i in datalist ]
    dates.sort()
    
    def parse_date(datestring):
    # Convert the date string into a pandas datetime object
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    # Defining the startdate and enddate from parse_date
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    # Set "index" as range from startdate to enddate by day
    index=pd.date_range(startdate, enddate, freq='D')
    # Define dataframe within index and columns
    vaccinatedpeopledf=pd.DataFrame(index=index, columns=['cumPeopleVaccinatedFirstDoseByPublishDate', 'cumPeopleVaccinatedSecondDoseByPublishDate', 'cumPeopleVaccinatedThirdInjectionByPublishDate'])

    for i in datalist: # each "i" is a dictionary with date, cumPeopleVaccinatedFirstDoseByPublishDate, cumPeopleVaccinatedSecondDoseByPublishDate and cumPeopleVaccinatedThirdInjectionByPublishDate
        date=parse_date(i['date'])
        for column in ['cumPeopleVaccinatedFirstDoseByPublishDate', 'cumPeopleVaccinatedSecondDoseByPublishDate', 'cumPeopleVaccinatedThirdInjectionByPublishDate']:
            if pd.isna(vaccinatedpeopledf.loc[date, column]): 
                # replace None with 0 in our data 
                value=float(i[column]) if i[column]!=None else 0
                # access a specific location in the dataframe using .loc
                # and put index,column in a single set of [ ]
                vaccinatedpeopledf.loc[date, column]=value
            
    # fill null value with 0
    vaccinatedpeopledf.fillna(0, inplace=True)
    
    return vaccinatedpeopledf

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
vaccinatedpeopledf=wrangle_data(data) # vaccinatedpeopledfdf is the dataframe for plotting

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [5]:
# Create a refresh button which will refresh the graph when clicked
def access_api():
    
    filters=[
    'areaType=Region',
    'areaName=London'
    ]
    
    structure={
        "date": "date",
        "cumPeopleVaccinatedFirstDoseByPublishDate": "cumPeopleVaccinatedFirstDoseByPublishDate",
        "cumPeopleVaccinatedSecondDoseByPublishDate": "cumPeopleVaccinatedSecondDoseByPublishDate",
        "cumPeopleVaccinatedThirdInjectionByPublishDate": "cumPeopleVaccinatedThirdInjectionByPublishDate" 
    }
    
    api=Cov19API(filters=filters, structure=structure)
    
    vaccinatedpeople=api.get_json()
    
    return data # Return data read from the API

In [6]:
def api_button_callback(button):
    # Get raw data from the API
    apidata=access_api()
    # Wrangle the data and overwrite the dataframe for plotting using global df
    global df
    # Call a function wrangle_data and pass apidata
    df=wrangle_data(apidata)
    refresh_graph(series.value, scale.value)
    # Change the icon to refresh
    apibutton.icon="refresh"
    # Button is active and can be clicked as many as user want
    apibutton.disabled=False

apibutton=wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='warning',
    tooltip="Click to download current Public Health London data",
    icon='exclamation-triangle'
)

# Register button_callback function, name of the function inside the brackets, with the button
apibutton.on_click(api_button_callback) 

display(apibutton)

Button(button_style='warning', description='Refresh Data', icon='exclamation-triangle', style=ButtonStyle(), t…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [7]:
# Set a multiple selection widget for series
series=wdg.SelectMultiple(
    options=['cumPeopleVaccinatedFirstDoseByPublishDate', 'cumPeopleVaccinatedSecondDoseByPublishDate', 'cumPeopleVaccinatedThirdInjectionByPublishDate'],
    value=['cumPeopleVaccinatedFirstDoseByPublishDate', 'cumPeopleVaccinatedSecondDoseByPublishDate', 'cumPeopleVaccinatedThirdInjectionByPublishDate'],
    rows=3,
    description='Stats:',
    disabled=False
)

# Set a radio button widget for scale
scale=wdg.RadioButtons(
    options=['linear', 'log'],
    value='linear', 
    description='Scale:',
    disabled=False
)

# Try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

# Create a function for refresh the data
def refresh_graph(gcols, gscale):
    series.value=['cumPeopleVaccinatedFirstDoseByPublishDate', 'cumPeopleVaccinatedSecondDoseByPublishDate', 'cumPeopleVaccinatedThirdInjectionByPublishDate'] # Back to the default
    scale.value='linear'

# Callback function
def vaccinatedpeople_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        vaccinatedpeopledf[list(gcols)].plot(logy=logscale)
        plt.title('CumPeopleVaccinated') # Put title to graph
        plt.xlabel('Date') # Put xlabel to graph
        plt.ylabel('Number of Vaccinated People') # Put ylabel to graph
        plt.show() # To update the graph
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    

# Keep calling vaccinatedpeople_graph(gcols=value_of_series, gscale=value_of_scale); 
# Capture output in widget graph   
graph=wdg.interactive_output(vaccinatedpeople_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember that if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).